In [21]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima
import db_connection as dbcon

#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# parameters
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')

# connect to db
db = dbcon.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

DB Credentials from file
DB connected successfully!!!


In [34]:
MAX_GIST_TO_COMPUTE = 10 # to limit the ouput of the query
query = {
    "$and" : [
        #{ "name":"Spain" },
        #{"gist": {"$exists": True} },
        {"gist": {"$exists": False} }
    ]  
}
images_metadata = [img_metadata for img_metadata in images_lib_col.find(query)]
images_filenames = [image["filename"] for image in images_metadata[:MAX_GIST_TO_COMPUTE]]
print(len(images_filenames))
for image in images_metadata[:10]:
    print(image["filename"])

10
29.6502_-95.3457_17_600.png
29.8059_-95.5329_17_600.png
29.7646_-95.5376_13_600.png
29.7369_-95.3517_15_600.png
29.7459_-95.432_13_600.png
29.5293_-95.429_10_600.png
29.91_-95.5629_13_600.png
29.8515_-95.3965_17_600.png
42.1919_-71.0078_14_600.png
42.5074_-71.1973_10_600.png


In [35]:
%%time
gist_uploaded, gist_vectors = ima.gist_calculate_and_load(images_filenames, IMG_FOLDER, images_lib_col)
print(gist_uploaded, "gist values uploaded")

10 gist values uploaded
CPU times: user 14.4 s, sys: 581 ms, total: 15 s
Wall time: 15.7 s
